In [1]:
import numpy as np
import pandas as pd
import warnings
import pickle
import arff
import random

from missforest.missforest import MissForest
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

In [2]:
# Set seed for Python's random module
random.seed(42)
# Set seed for NumPy
np.random.seed(42)

In [3]:
with open("C:/Users/marti/Desktop/DNN_missing_data/public_procurement/dataset.arff", 'r', encoding="utf-8") as f:
    dataset = arff.load(f)

df = pd.DataFrame(dataset['data'], columns=[attr[0] for attr in dataset['attributes']])

In [4]:
Y = df['award_value_euro']
Y = np.log(Y - np.min(Y) + 1)

df_numerical = df.select_dtypes(include=['number'])
Z = df_numerical.drop('award_value_euro', axis=1)
Omega = Z.notna()

In [5]:
# Mean imputation
Z_MI = Z.fillna(Z.mean())
scaler = StandardScaler()
Z_MI = pd.DataFrame(scaler.fit_transform(Z_MI), columns=Z_MI.columns)

In [6]:
# Missforest imputation
MF_imputer = MissForest(RandomForestRegressor(n_estimators=10, n_jobs=-1), early_stopping=True)
Z_MF = MF_imputer.fit_transform(Z)
scaler = StandardScaler()
Z_MF = pd.DataFrame(scaler.fit_transform(Z_MF), columns=Z_MF.columns)

c:\Users\marti\AppData\Local\Programs\Python\Python313\Lib\site-packages\missforest\missforest.py:333: UserWarning: Label encoding is no longer performed by default. Users will have to perform categorical features encoding by themselves.
  warnings.warn("Label encoding is no longer performed by default. "
 40%|████      | 2/5 [01:58<03:00, 60.25s/it]c:\Users\marti\AppData\Local\Programs\Python\Python313\Lib\site-packages\missforest\missforest.py:303: UserWarning: NRMSE increased.
  warnings.warn("NRMSE increased.")
c:\Users\marti\AppData\Local\Programs\Python\Python313\Lib\site-packages\missforest\missforest.py:453: UserWarning: Stopping criterion triggered during fitting. Before last imputation matrix will be returned.
  warnings.warn(
 40%|████      | 2/5 [03:03<04:34, 91.59s/it]
c:\Users\marti\AppData\Local\Programs\Python\Python313\Lib\site-packages\missforest\missforest.py:490: UserWarning: Label encoding is no longer performed by default. Users will have to perform categorical fe

In [7]:
# Iterative imputer
II_imputer = IterativeImputer(max_iter=5)
Z_II = II_imputer.fit_transform(Z)
scaler = StandardScaler()
Z_II = pd.DataFrame(scaler.fit_transform(Z_II), columns=Z.columns)

c:\Users\marti\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [8]:
# Store the data in public_procurement_data.pkl
public_procurement_data = {'Z_MI': Z_MI,
                     'Z_MF': Z_MF,
                     'Z_II': Z_II,
                     'Omega': Omega,
                     'Y': Y}

with open("C:/Users/marti/Desktop/DNN_missing_data/public_procurement/public_procurement_data.pkl", "wb") as f:
    pickle.dump(public_procurement_data, f)